# companies

> Set of functions for dealing with Copper companies

In [ ]:
#| default_exp companies

In [9]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
from cuapi_wrapper import core
from cuapi_wrapper.core import set_headers
import pandas as pd
import pytz
from tqdm import tqdm

In [2]:
#| export
def prc_clean_company_data(company_data:list, #Returned company data as list of dictionaries,
                          cf_fields:list = [], # list of custom fields on companies you would like to include
):
    """Process to clean returned company data"""

    core.prc_cf_dicts()

    global custom_fields
    custom_fields = core.get_global_var('custom_fields')
    custom_fields_dict = core.get_global_var('custom_fields_dict')
    

    def clean_date(date):
        try:
            if date is None:
                return None  # Handle None values (empty rows)

            if isinstance(date, int) and len(str(date)) >= 6 and len(str(date)) <= 10:
                # If it's an integer with <= 10 digits, assume it's a Unix timestamp
                ct_timezone = pytz.timezone('US/Central')
                new_date = pd.to_datetime(date, unit='s',utc=True).tz_convert(ct_timezone)
                return new_date
            else:
                return date
        except Exception as e:
            return date

    native_items = ['id', 'name', 'address', 'assignee_id', 'contact_type_id']

    output_dict = {}

    for item in native_items:
        output_dict[item] = company_data.get(item, None)
    
    custom_field_data = company_data['custom_fields']

    for dict_item in custom_field_data:
        item_id = dict_item['custom_field_definition_id']
        item_name = custom_fields_dict.get(item_id, None)
        data_type = custom_fields[item_id].get('data_type')

        if item_name not in cf_fields or item_name is None:
            continue
        elif item_name is not None and ('options' in list(custom_fields[item_id].keys())):
            item_value = dict_item['value']
            value_name = core.cf_option_name(item_id,item_value)
            output_dict[item_name] = value_name
        elif data_type == 'Date':
            cleaned_date = clean_date(dict_item['value'])
            output_dict[item_name] = cleaned_date
        else:
            item_value = dict_item['value']
            output_dict[item_name] = item_value

    return output_dict

In [7]:
#| export
def search(search_params:dict={}, # search params for standard company fields
            cf_search:list=[],# search params for custom fields
            clean_data:bool = True, # Whether to clean results or not
            cf_fields:list = [], # list of custom fields on companies you would like to include
            drop_cols:list = None, # Columns to drop
            )->pd.DataFrame:
    """Searches copper companies and returns Pandas DataFrame"""

    global copper_headers

    if "copper_headers" not in globals() and core.get_global_var('copper_headers') is None:
        raise ValueError('header information must be set with set_headers(). \nSee help(core.set_headers) for more info.')
    
    if "copper_headers" not in globals() and core.get_global_var('copper_headers') is not None:
        copper_headers = core.get_global_var('copper_headers')
    
    total_pages = page = 1
    combined_results = []
    
    Sess = core.get_session(copper_headers)

    while page <= total_pages:
        page_params = {
            "page_size": 100,
            "page_number": page,
            }
        
        if search_params != {}: page_params.update(search_params)

        if cf_search != {}:
            cf_addition = {"custom_fields":cf_search}
            page_params.update(cf_addition) 

        result = Sess.post('https://api.copper.com/developer_api/v1/companies/search',json=page_params)
    
        if result.status_code == 200:
            total_pages = (int(result.headers['X-PW-TOTAL'])//100)+1
            
            # Creatig Progress Bar:
            if page == 1: progress_bar = tqdm(total=total_pages,desc='Searching Copper')
            progress_bar.update(1)  # Update the progress bar
            
            result_json = result.json()
            combined_results.extend(result_json)
            page +=1

        else:
            print(f"Issue with page {page}. Stopping.")
            break

    progress_bar.close()  # Close the progress bar when done

    if not clean_data:
        print('Returning raw results')
        return combined_results
 
   
    print('\nData Recieved. Cleaning results:')

    cleaned_data = [prc_clean_company_data(result, cf_fields) for result in combined_results]
    cleaned_data_df = pd.DataFrame(cleaned_data)

    print('Returning cleaned results')
    if isinstance(drop_cols,list):      return cleaned_data_df.drop(columns=drop_cols,inplace=True)
    else:                               return cleaned_data_df 

### Searching Copper Companies

Working with Copper Companies is a core feature of cuapi_wrapper and is done with the `companies.search` function.

In [12]:
show_doc(search)

---

[source](https://github.com/cooper-richason/cuapi-wrapper/blob/main/cuapi_wrapper/companies.py#L85){target="_blank" style="float:right; font-size:smaller"}

### search

>      search (search_params:dict={}, cf_search:list=[], clean_data:bool=True,
>              cf_fields:list=[], drop_cols:list=None)

Searches copper companies and returns Pandas DataFrame

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| search_params | dict | {} | search params for standard company fields |
| cf_search | list | [] | search params for custom fields |
| clean_data | bool | True | Whether to clean results or not |
| cf_fields | list | [] | list of custom fields on companies you would like to include |
| drop_cols | list | None | Columns to drop |
| **Returns** | **DataFrame** |  |  |

#### API Resonse Cleaning

By default, `companies.search` cleans the response data from Copper using `prc_clean_company_data`. This process does several key things:

1. 


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()